In [14]:
!pip install xgboost
!pip install deap

   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.5/150.0 MB 1.3 MB/s eta 0:01:57
   ---------------------------------------- 0.8/150.0 MB 1.4 MB/s eta 0:01:47
   ---------------------------------------- 1.3/150.0 MB 1.8 MB/s eta 0:01:23
    --------------------------------------- 2.1/150.0 MB 2.4 MB/s eta 0:01:02
    --------------------------------------- 3.1/150.0 MB 3.0 MB/s eta 0:00:49
   - -------------------------------------- 4.5/150.0 MB 3.7 MB/s eta 0:00:40
   - -------------------------------------- 6.8/150.0 MB 4.9 MB/s eta 0:00:30
   -- ------------------------------------- 9.7/150.0 MB 5.8 MB/s eta 0:00:25
   --- ------------------------------------ 13.1/150.0 MB 7.0 MB/s eta 0:00:20
   ---- -----


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:

import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from deap import base, creator, tools, algorithms
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.feature_selection import SelectFromModel
import xgboost as xgb
from xgboost import XGBClassifier


In [2]:
# === 1. Load Dataset ===
df = pd.read_csv("Lung Cancer Dataset.csv")  # Sesuaikan dengan nama file dataset Anda
df['PULMONARY_DISEASE'] = df['PULMONARY_DISEASE'].map({'YES': 1, 'NO': 0})  # Encoding target variable


In [33]:
# === 2. Parameter Genetic Algorithm ===
POPULATION_SIZE = 20  # Ukuran populasi
N_GENERATIONS = 50  # Jumlah iterasi generasi
MUTATION_RATE = 0.2  # Peluang mutasi

In [34]:
#=== 3. Inisialisasi Genetic Algorithm ===
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_bool", np.random.randint, 0, 2)  # 0 = fitur tidak dipilih, 1 = fitur dipilih
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=len(X.columns))
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [35]:
# === 4. Fungsi Evaluasi ===
def evaluate(individual):
    selected_features = [col for i, col in enumerate(X.columns) if individual[i] == 1]

    if len(selected_features) == 0:
        return 0,  # Jika tidak ada fitur yang dipilih, akurasi 0

    X_selected = X[selected_features]

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

    # Random Forest model
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')

    return np.mean(scores),  # Mengembalikan rata-rata akurasi

In [36]:
# === 5. Operator GA ===
toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)  # Crossover: Kombinasi dua individu
toolbox.register("mutate", tools.mutFlipBit, indpb=MUTATION_RATE)  # Mutasi
toolbox.register("select", tools.selTournament, tournsize=3)  # Seleksi turnamen

In [38]:
# === 6. Jalankan Genetic Algorithm ===
population = toolbox.population(n=POPULATION_SIZE)
hof = tools.HallOfFame(1)  # Menyimpan individu terbaik
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=N_GENERATIONS, stats=None, halloffame=hof, verbose=True)

gen	nevals
0  	20    
1  	10    
2  	7     
3  	6     
4  	11    
5  	15    
6  	12    
7  	10    
8  	13    
9  	14    
10 	10    
11 	10    
12 	13    
13 	10    
14 	13    
15 	13    
16 	18    
17 	14    
18 	13    
19 	15    
20 	9     
21 	5     
22 	7     
23 	9     
24 	11    
25 	15    
26 	15    
27 	10    
28 	10    
29 	12    
30 	16    
31 	9     
32 	14    
33 	16    
34 	11    
35 	11    
36 	19    
37 	12    
38 	11    
39 	8     
40 	16    
41 	12    
42 	11    
43 	11    
44 	8     
45 	14    
46 	13    
47 	13    
48 	14    
49 	6     
50 	14    


([[0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1],
  [0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1],
  [0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1],
  [0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1],
  [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1],
  [1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1],
  [0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1],
  [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1],
  [0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1],
  [0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1],
  [0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1],
  [0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1],
  [0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0],
  [0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1],
  [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1],
  [0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1],
  [0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1],
  [0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1],
  [0, 0, 1

In [39]:
# === 7. Tampilkan Fitur Terbaik ===
best_individual = hof[0]
best_features = [col for i, col in enumerate(X.columns) if best_individual[i] == 1]

print("Fitur Terbaik yang Dipilih oleh Genetic Algorithm:", best_features)

Fitur Terbaik yang Dipilih oleh Genetic Algorithm: ['SMOKING', 'FINGER_DISCOLORATION', 'MENTAL_STRESS', 'EXPOSURE_TO_POLLUTION', 'LONG_TERM_ILLNESS', 'ENERGY_LEVEL', 'IMMUNE_WEAKNESS', 'BREATHING_ISSUE', 'ALCOHOL_CONSUMPTION', 'THROAT_DISCOMFORT', 'OXYGEN_SATURATION', 'FAMILY_HISTORY', 'STRESS_IMMUNE']


In [40]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import xgboost as xgb
from xgboost import XGBClassifier
from deap import base, creator, tools, algorithms

In [41]:

# Load dataset (Ganti 'dataset.csv' dengan nama file yang sesuai)
df = pd.read_csv("Lung Cancer Dataset.csv")

df['PULMONARY_DISEASE'] = df['PULMONARY_DISEASE'].map({'YES': 1, 'NO': 0})  # Encoding target variable

# Pisahkan fitur dan target
X = df.drop(columns=['PULMONARY_DISEASE'])
y = df['PULMONARY_DISEASE']

# Konversi fitur kategori ke numerik (jika ada)
X = pd.get_dummies(X)


In [42]:

# Split data hanya sekali
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Simpan hasil awal sebelum GA
result = []

In [44]:

# Train Random Forest sebelum GA
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
result.append(["Random Forest Sebelum GA", 
                accuracy_score(y_test, y_pred_rf),
                precision_score(y_test, y_pred_rf),
                recall_score(y_test, y_pred_rf),
                f1_score(y_test, y_pred_rf),
                roc_auc_score(y_test, y_pred_rf)])

# Train XGBoost sebelum GA
xgb_model = XGBClassifier(n_estimators=100, random_state=42,  eval_metric="logloss")
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)
result.append(["XGBoost Sebelum GA", 
                accuracy_score(y_test, y_pred_xgb),
                precision_score(y_test, y_pred_xgb),
                recall_score(y_test, y_pred_xgb),
                f1_score(y_test, y_pred_xgb),
                roc_auc_score(y_test, y_pred_xgb)])